In [8]:
import os
os.system("pip install gensim matplotlib scikit-learn pandas")

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 26.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 9.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 11.9 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


0

In [9]:
import gensim
import gensim.downloader
import gensim.downloader as api

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA

In [5]:
model_name = "english_word2vec.bin"
model_path = os.path.join("..","in","model",model_name)
model = gensim.models.KeyedVectors.load(model_path)

In [11]:
#data
data_name = "data_spoty.csv"
data_path = os.path.join("..","in","data",data_name)
data = pd.read_csv(data_path)

In [15]:
#subset
singer = "ABBA"
sub_data = data.loc[data['artist'] == singer]

In [17]:
#embed
query_word = "love"
n_embeds = 12
query_array = model.most_similar(query_word,topn=n_similar)
query_array.append((query_word, 1))

In [21]:
# find in subset
for word_value_pair in query_array:
    target_word = word_value_pair[0]

    for lyric_words in sub_data[lyric_words]


    


KeyError: "None of [Index([0, 2], dtype='int64')] are in the [columns]"